## ConversationalRetrievalChain

*   conversation memory + RetrievalQAChain
*   Allow for passing in conversation history which can be used for follow up questions.



In [ ]:
import os
os.environ["OPENAI_API_KEY"] = " "

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# load document
loader = PyPDFLoader("file_name.pdf")
documents = loader.load()
# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
# text-embedding-ada-002
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2}) # top 2 documnets
# create a chain to answer questions
qa = ConversationalRetrievalChain.from_llm(OpenAI(), retriever) # text-davinci-003 (GPT-3)
chat_history = []

In [ ]:
query = "what is the overall summary of this conversation ?"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result["answer"]

' This conversation is about a data analytics platform that uses AI and machine learning to provide real-time insights. It has features like advanced data visualization tools, deep learning algorithms, and robust APIs for integration. It also uses parallel processing and distributed computing to handle large and intricate datasets.'

In [ ]:
chat_history = [(query, result["answer"])]
query = "How is the agent attitude towards the customer?"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result["answer"]

' The agent has a helpful and patient attitude towards the customer.'

In [ ]:
chat_history = [(query, result["answer"])]
query = "Is the agent was aggressive while talking to customer ?"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result["answer"]

' No, the agent was not aggressive while talking to the customer.'